In [1]:
%load_ext autoreload
%autoreload 2

# Mongo DB connection Prototyping

In [2]:
# imports and config
import pandas as pd
from pymongo import UpdateOne, MongoClient
import numpy as np
from datetime import datetime
import funcy
import json

# siblings
import data_loader # !symlink hack to make this work - ln -s ../app/data_loader.py! 

In [3]:
def get_mongo_client(mongo_connect_dict, tls_flag=True, tlsAllowInvalidCertificates=False):
    return MongoClient(host=mongo_connect_dict['host'],
                            port=int(mongo_connect_dict['port']),
                            username=mongo_connect_dict['user'],
                            password=mongo_connect_dict['pass'],
                            authSource=mongo_connect_dict['auth_source'],
                            tls=tls_flag,
                            tlsAllowInvalidCertificates=tlsAllowInvalidCertificates)


def print_collection_sizes(notes_db):
    for coll_name in notes_db.list_collection_names():
        if coll_name != 'system.views':
            if coll_name.startswith('v_'):
                print(f"VIEW      : {coll_name}                has {notes_db[coll_name].count_documents({})} documents")
            else:
                print(f"COLLECTION: {coll_name}                has {notes_db[coll_name].estimated_document_count()} documents")
        else:
            ...

In [4]:
# creating the client
local_connect_dict = {
    "host":"localhost",
    "port":27017,
    "user":'',
    "pass":'',
    "auth_source":''
}
mongo_client = get_mongo_client(local_connect_dict, tls_flag=False, tlsAllowInvalidCertificates=True)

"""
Test client by reading out current database names
"""
print(mongo_client.list_database_names())
if 'pv_db' in mongo_client.list_database_names():
    print("Database already exists")
    pv_db = mongo_client['pv_db']
    print(print_collection_sizes(pv_db))
else:
    print('Creating pv_db database')
    pv_db = mongo_client['pv_db']
    pv_db.create_collection('projects')
    pv_db.create_collection('users')

['admin', 'config', 'local', 'pv_db']
Database already exists
COLLECTION: projects                has 4 documents
COLLECTION: users                has 2 documents
COLLECTION: project_data                has 39 documents
None


In [6]:
# user data
pv_dl = data_loader.MongoDataLoader(local_connect_dict, 'pv_db','users','projects','project_data' )
users = pv_dl.get_user_data()
print(len(users))
print(users[:5])
with open('../example_database/users.json', 'w') as user_json_out:
    json.dump(users, user_json_out)

2
[{'_id': 'awriedl', 'email': 'awriedl@ucdavis.edu', 'print_name': 'Bill', 'user_name': 'awriedl'}, {'_id': 'pieval', 'email': 'pieval@pieval.edu', 'print_name': 'pieval', 'user_name': 'pieval'}]


In [12]:
# project data
pv_projects = pv_dl.get_projects()
print(set([x['project_name'] for x in pv_projects]))
print(len(pv_projects))
print(pv_projects[:5])
with open('../example_database/projects.json', 'w') as proj_json_out:
    json.dump(pv_projects, proj_json_out)

{'cncr_hist_mc_demo', 'kappa_lambda_demo', 'movie_reviews_demo', 'image_demo'}
4
[{'_id': 'kappa_lambda_demo', 'class_list': None, 'data_type': 'text', 'project_description': 'DEMO PROJECT: Determining if monotypic/clonal plasma cell populations exist', 'project_mode': 'binary', 'project_name': 'kappa_lambda_demo', 'user_list': ['awriedl', 'pieval']}, {'_id': 'cncr_hist_mc_demo', 'class_list': ['carcinoma', 'adenocarcinoma', 'squamous cell', 'small cell', 'hodgkins lymphoma', 'myeloma', 'none'], 'data_type': 'text', 'project_description': 'DEMO PROJECT: Classifying cancer histology from pathology report text', 'project_mode': 'multiclass', 'project_name': 'cncr_hist_mc_demo', 'user_list': ['awriedl', 'pieval']}, {'_id': 'movie_reviews_demo', 'class_list': ['positive', 'bubbly', 'neutral', 'dismal', 'negative'], 'data_type': 'text', 'project_description': 'DEMO PROJECT: Classifying sentiment from movie reviews', 'project_mode': 'multiclass', 'project_name': 'movie_reviews_demo', 'user_l

In [14]:
# project data data
pv_project_data = pv_dl.get_project_data()
print(set([x['project_name'] for x in pv_project_data]))
print(len(pv_project_data))
print(pv_project_data[:5])
with open('../example_database/project_data.json', 'w') as proj_data_json_out:
    json.dump(pv_project_data, proj_data_json_out)

{'kappa_lambda_demo', 'image_demo', 'cncr_hist_mc_demo', 'movie_reviews_demo', 'context_demo'}
39
[{'_id': 'movie_reviews_demo_0', 'data': 'Based on an actual story, John Boorman shows the struggle of an American doctor, whose husband and son were murdered and she was continually plagued with her loss. A holiday to Burma with her sister seemed like a good idea to get away from it all, but when her passport was stolen in Rangoon, she could not leave the country with her sister, and was forced to stay back until she could get I.D. papers from the American embassy. To fill in a day before she could fly out, she took a trip into the countryside with a tour guide. "I tried finding something in those stone statues, but nothing stirred in me. I was stone myself." <br /><br />Suddenly all hell broke loose and she was caught in a political revolt. Just when it looked like she had escaped and safely boarded a train, she saw her tour guide get beaten and shot. In a split second she decided to jum

## Project Data and Annotations

In [ ]:
prj_data = pv_dl.get_project_data(project_name='cncr_hist_mc_demo')
print(len(prj_data))


# convert raw project data (maybe annots maybe not) into dataframe
prj_data_df = pd.DataFrame(prj_data)
print(prj_data_df.shape)
display(prj_data_df)

if 'annots' in prj_data_df.columns:
    prj_data_annots = prj_data_df.explode('annots').reset_index(drop=True)
    print(prj_data_annots.shape)
    display(prj_data_annots)
    prj_data_annots_final = prj_data_annots.join(prj_data_annots['annots'].apply(pd.Series))
    print(prj_data_annots_final.shape)
    display(prj_data_annots_final)
else:
    print("No annots for this project")

### Adding/Deleting Annots

In [ ]:
# adding an annotation
# step 1 - obtain the example
one_example = prj_data[1]
cur_dt = datetime.now()
cur_dt = cur_dt.replace(microsecond=0)
# step 2 - see if any prior annotations exists
if 'annots' in one_example:
    print("There are prior annotations!")
    # in this case grab the current annot array
    annot_array = one_example['annots']
    max_annot_id = max([x['annot_id'] for x in annot_array])
    one_annot = {
        'annot_id':max_annot_id+1,
        'response_time':cur_dt,
        'user_name':'pieval_test',
        'user_ip':'127.0.0.1',
        'response':'testing',
        'context_viewed':'yes'
    }
    annot_array.append(one_annot)
else:
    print("There are no prior annotations")
    # in this case create a new annot array
    one_annot = {
        'annot_id':0,
        'response_time':cur_dt,
        'user_name':'pieval_test',
        'user_ip':'127.0.0.1',
        'response':'testing',
        'context_viewed':'yes'
    }    
    annot_array = [one_annot]

extra_feature_dict = {'annots':annot_array}
pv_dl.update_example(one_example, extra_feature_dict)

In [ ]:
## deleting an annotation
annot_id_to_delete = 2
one_example = prj_data[0]
if 'annots' in one_example:
    print(f"There are prior annotations! - deleting annot with id = {annot_id_to_delete}")
    annot_array = one_example['annots']
    new_annot_array = [x for x in annot_array if x['annot_id'] != annot_id_to_delete]
else:
    print("There are no prior annotations - Nothing to delete!")

print("OG annot array")
print(annot_array)
print("updated annot array")
print(new_annot_array)

# now set the version in mongo to the updated annot array
extra_feature_dict = {'annots':new_annot_array}
pv_dl.saveAnnot(one_example, extra_feature_dict)

## Loading/reformatting Example data into JSON

In [ ]:
proj_classes_df = pd.read_csv('../example_database/project_classes.csv')
proj_users = pd.read_csv('../example_database/project_users.csv')
proj_df = pd.read_csv('../example_database/projects.csv')

# project data - the documents
proj_data_df = pd.read_csv('../example_database/project_data.csv')
# user information
user_df = pd.read_csv('../example_database/user_details.csv')

# these data WONT be loaded in to the example database - this structure will be considered completely legacy
annot_df = pd.read_csv('../example_database/annotation_events.csv')

### Merging project data together

In [ ]:
proj_users_grp = (proj_users.groupby('project_name')
                  .agg(
                      user_list=('user_name',lambda x:list(x))
                      )
                    .reset_index()
                )

proj_classes_grp = (proj_classes_df.groupby('project_name')
                  .agg(
                      class_list=('class',lambda x:list(x))
                      )
                    .reset_index()
                )

proj_final_df = pd.merge(proj_df, proj_users_grp, how='left', on='project_name')
proj_final_df = pd.merge(proj_final_df, proj_classes_grp, how='left', on='project_name')

proj_final_df

In [ ]:
# write project data to mongo
proj_dict_list = proj_final_df.to_dict(orient='records')
for one_obj in proj_dict_list:
    for k,v in one_obj.items():
        if isinstance(v, (pd._libs.tslibs.nattype.NaTType)):
            one_obj[k] = None
        if (isinstance(v, float) and np.isnan(v)):
            one_obj[k] = None
        if isinstance(v, np.ndarray):
            one_obj[k] = v.tolist()

proj_upserts = [UpdateOne({"_id": str(x.get('project_name'))},
                    {"$set": x},
                    upsert=True) for x in proj_dict_list]

In [ ]:
bulk_res = pv_db['projects'].bulk_write(proj_upserts)
bulk_api_result_dict = bulk_res.bulk_api_result
bulk_api_result_dict

### Project Data

In [ ]:
proj_data_df["_id"] = proj_data_df['project_name'] + '_' + proj_data_df['example_id'].astype(str)
proj_data_df

In [ ]:
proj_data_df['_id'].is_unique

In [ ]:
# write project data to mongo
proj_data_dict_list = proj_data_df.to_dict(orient='records')
for one_obj in proj_data_dict_list:
    for k,v in one_obj.items():
        if isinstance(v, (pd._libs.tslibs.nattype.NaTType)):
            one_obj[k] = None
        if (isinstance(v, float) and np.isnan(v)):
            one_obj[k] = None
        if isinstance(v, np.ndarray):
            one_obj[k] = v.tolist()

proj_data_upserts = [UpdateOne({"_id": str(x.get('_id'))},
                    {"$set": x},
                    upsert=True) for x in proj_data_dict_list]

In [ ]:
bulk_res = pv_db['project_data'].bulk_write(proj_data_upserts)
bulk_api_result_dict = bulk_res.bulk_api_result
bulk_api_result_dict

### User Data

In [ ]:
user_df["_id"] = user_df['user_name']
user_df

In [ ]:
# write project data to mongo
user_dict_list = user_df.to_dict(orient='records')
for one_obj in user_dict_list:
    for k,v in one_obj.items():
        if isinstance(v, (pd._libs.tslibs.nattype.NaTType)):
            one_obj[k] = None
        if (isinstance(v, float) and np.isnan(v)):
            one_obj[k] = None
        if isinstance(v, np.ndarray):
            one_obj[k] = v.tolist()

user_upserts = [UpdateOne({"_id": str(x.get('_id'))},
                    {"$set": x},
                    upsert=True) for x in user_dict_list]
user_upserts

In [ ]:
bulk_res = pv_db['users'].bulk_write(user_upserts)
bulk_api_result_dict = bulk_res.bulk_api_result
bulk_api_result_dict

## Annotation Data

> NOTE!  In the mongo version of this project, there will not be a seperate annoation collection.  We will simply update the original documents in project_data with additional k:v pairs resulting from the annotation process

In [ ]:
annot_df